In [1]:
import spacy
from spacy.tokens import DocBin
from spacy.training import Example
from thinc.optimizers import Adam
from tqdm import tqdm
import json

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
# Add the NER pipeline if it's not present
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe("ner")

# Add your custom labels to the NER component
labels = ["GPE", "DISASTER"]
for label in labels:
    ner.add_label(label)

# Function to load data from the .jsonl file
def load_training_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    training_data = []
    for line in lines:
        entry = json.loads(line)
        text = entry[0]
        entities = entry[1]["entities"]
        training_data.append((text, {"entities": entities}))
    return training_data

# Convert training data into spaCy's Example objects
def create_training_examples(nlp, train_data):
    examples = []
    for text, annotations in tqdm(train_data):
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        examples.append(example)
    return examples

In [9]:

# Load your custom NER training data
train_data = load_training_data("train/data3.jsonl")

# Convert training data to spaCy's Example format
train_examples = create_training_examples(nlp, train_data[:1000])

100%|██████████| 1000/1000 [00:01<00:00, 989.34it/s]


In [10]:
learning_rate = 0.000001  # Change this to your desired value

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.resume_training()
    nlp.config["training"]["optimizer"]["learn_rate"] = learning_rate
    # Initialize the optimizer with the NER component
    # optimizer = nlp.create_optimizer()  # Just create the optimizer without any parameters
    # nlp.config["training"]["optimizer"]["learn_rate"] = learning_rate
    # Define your custom learning rate and batch size

    # Fine-tune the model on the new dataset
    for iteration in range(20):  # Adjust the number of iterations as needed
        losses = {}
        # Create batches with the specified batch size
        # batches = spacy.util.minibatch(train_examples, size=spacy.util.compounding(4.0, 32.0, 1.001))
        batches = spacy.util.minibatch(train_examples, size=spacy.util.compounding(4.0, 32.0, 1.001))
        for batch in batches:
            # Update the model with a custom learning rate
            nlp.update(batch, sgd=optimizer, drop=0.35, losses=losses)
        print(f"Iteration {iteration + 1}, Losses: {losses}")

    # Save the updated model
nlp.to_disk("0fner_model_custom_4_1")

print("Model fine-tuning complete!")

Iteration 1, Losses: {'ner': 1692.1467939288493}
Iteration 2, Losses: {'ner': 499.4314064113034}
Iteration 3, Losses: {'ner': 286.63393818879007}
Iteration 4, Losses: {'ner': 197.08769977499125}
Iteration 5, Losses: {'ner': 148.9309282132022}
Iteration 6, Losses: {'ner': 85.0569526385531}
Iteration 7, Losses: {'ner': 57.29338382675076}
Iteration 8, Losses: {'ner': 73.48997945942445}
Iteration 9, Losses: {'ner': 80.21941100280584}
Iteration 10, Losses: {'ner': 59.791510166100345}
Iteration 11, Losses: {'ner': 45.95952556672817}
Iteration 12, Losses: {'ner': 49.282037949899}
Iteration 13, Losses: {'ner': 24.695770303232806}
Iteration 14, Losses: {'ner': 45.725618190981244}
Iteration 15, Losses: {'ner': 36.029180169302485}
Iteration 16, Losses: {'ner': 29.08204286306493}
Iteration 17, Losses: {'ner': 43.84203517078039}
Iteration 18, Losses: {'ner': 31.27555294935127}
Iteration 19, Losses: {'ner': 34.76315409491644}
Iteration 20, Losses: {'ner': 25.666622567994725}
Model fine-tuning comple

In [24]:
# Load the fine-tuned model
nlp_custom = spacy.load("0fner_model_custom_2_2")

# Test the model with a sample tweet
doc = nlp_custom("The aftershocks pattern in purple suggests that the M. Noto earthquake ruptured bilaterally, with the preliminary NEIC slip model in Japan also indicating slip on both sides of the epicenter in Ishikawa")
for ent in doc.ents:
    print(ent.text, ent.label_)


aftershocks DISASTER
M. Noto GPE
earthquake DISASTER
Japan GPE
epicenter DISASTER
Ishikawa GPE
